# 损失函数

## 自定义损失函数

"SMAPE loss" 是指 Symmetric Mean Absolute Percentage Error，中文意为对称平均绝对百分比误差。这是一种用于衡量预测模型精度的指标，通常用于评估时间序列预测模型的性能。

SMAPE 的计算公式如下：

\$
SMAPE = \frac{100\%}{n} \sum_{t=1}^{n} \frac{|F_t - A_t|}{(|F_t| + |A_t|)/2}
\$

其中：

- \(F_t\) 是模型预测的值；
- \(A_t\) 是实际观测到的值；
- \(n\) 是样本数量。

SMAPE 通常被用作损失函数，用于优化时间序列预测模型。在优化过程中，目标是尽量减小 SMAPE 损失，以提高模型的预测准确性。

In [1]:
"""
Loss functions for PyTorch.
"""

import torch as t
import torch.nn as nn
import numpy as np
import pdb


def divide_no_nan(a, b):
    """
    a/b where the resulted NaN or Inf are replaced by 0.
    """
    result = a / b
    result[result != result] = .0
    result[result == np.inf] = .0
    return result


class mape_loss(nn.Module):
    def __init__(self):
        super(mape_loss, self).__init__()

    def forward(self, insample: t.Tensor, freq: int,
                forecast: t.Tensor, target: t.Tensor, mask: t.Tensor) -> t.float:
        """
        MAPE loss as defined in: https://en.wikipedia.org/wiki/Mean_absolute_percentage_error

        :param forecast: Forecast values. Shape: batch, time
        :param target: Target values. Shape: batch, time
        :param mask: 0/1 mask. Shape: batch, time
        :return: Loss value
        """
        weights = divide_no_nan(mask, target)
        return t.mean(t.abs((forecast - target) * weights))


class smape_loss(nn.Module):
    def __init__(self):
        super(smape_loss, self).__init__()

    def forward(self, insample: t.Tensor, freq: int,
                forecast: t.Tensor, target: t.Tensor, mask: t.Tensor) -> t.float:
        """
        sMAPE loss as defined in https://robjhyndman.com/hyndsight/smape/ (Makridakis 1993)

        :param forecast: Forecast values. Shape: batch, time
        :param target: Target values. Shape: batch, time
        :param mask: 0/1 mask. Shape: batch, time
        :return: Loss value
        """
        return 200 * t.mean(divide_no_nan(t.abs(forecast - target),
                                          t.abs(forecast.data) + t.abs(target.data)) * mask)


class mase_loss(nn.Module):
    def __init__(self):
        super(mase_loss, self).__init__()

    def forward(self, insample: t.Tensor, freq: int,
                forecast: t.Tensor, target: t.Tensor, mask: t.Tensor) -> t.float:
        """
        MASE loss as defined in "Scaled Errors" https://robjhyndman.com/papers/mase.pdf

        :param insample: Insample values. Shape: batch, time_i
        :param freq: Frequency value
        :param forecast: Forecast values. Shape: batch, time_o
        :param target: Target values. Shape: batch, time_o
        :param mask: 0/1 mask. Shape: batch, time_o
        :return: Loss value
        """
        masep = t.mean(t.abs(insample[:, freq:] - insample[:, :-freq]), dim=1)
        masked_masep_inv = divide_no_nan(mask, masep[:, None])
        return t.mean(t.abs(target - forecast) * masked_masep_inv)


## nn.MSELoss

torch.nn.MSELoss(size_average=None, reduce=None, reduction="mean")

该函数默认用于计算两个输入对应元素差值平方和的均值。

如果同时给出了reduce、size_average、reduction三个参数，则首先看前两个参数。如果前两个参数均为None，则函数的返回值由reduction参数决定。如果前两个参数不全为None，则函数的返回值由前两个参数决定，在这种情况下，为None的那个参数默认为True。确定了三个参数的取值后，根据下述规则进行计算即可：

- 当reduce=True时，若size_average=True，则返回一个batch中所有样本损失的均值，结果为**标量**。注意，对于MESLoss函数来说，首先对该batch中的所有样本损失进行逐元素均值操作，然后对得到N个值再进行均值操作即得到返回值（假设批大小为N，即该batch中共有N个样本），用官网的话来说，就是The mean operation still operates over all the elements, and divides by N.
- 当reduce=True时，若size_average=False，则返回一个batch中所有样本损失的和，结果为**标量**。注意，对于MESLoss函数来说，首先对该batch中的所有样本损失进行逐元素求和操作，然后对得到N个值再进行求和操作即得到返回值（假设批大小为N，即该batch中共有N个样本），用官网的话来说，就是The sum operation still operates over all the elements.
- 当reduce=False时，则size_average参数失效，即无论size_average参数为False还是True，效果都是一样的。此时，函数返回的是一个batch中每个样本的损失，结果为**向量**。